(08:Continuous-integration-and-deployment)=
# Continuous integration and deployment
<hr style="height:1px;border:none;color:#666;background-color:#666;" />

If you've gotten this far, you now have a working knowledge of how to create a fully-featured Python package! We went through quite a lot to get here: we developed the source code for a Python package, learned about package structure, wrote documentation, created tests, and went through the process of releasing a new version of a package.

As you continue to develop your package into the future it would be helpful to automate many of these workflows so you and your collaborators can focus more on writing code and less on the nuances of packaging and testing. This is where **continuous integration** and **continuous deployment** come in (CI/CD)! The term CI/CD generally refers to the automated testing, building, and deployment of software. In this chapter we'll first walk through setting up CI and CD for a Python package, with the help of GitHub Actions. After that, we'll take our pipeline for a test run by adding a new feature to the `partypy` package we've been developing throughout this book. 

```{note}
The CD part of CI/CD is also sometimes referred to as "continuous delivery". Continuous delivery and continuous deployment have slightly different definitions: continuous delivery refers to preparing software for manual release by the developer, whereas continuous deployment takes this one step further and automates the release process too. We'll be referring to "continuous deployment" in this chapter.
```

## CI/CD tools

You could manually write and execute a CI/CD workflow by, for example, writing scripts that execute all of the steps we've walked through in previous chapters (i.e., running tests, building documented, build package, etc.). However, this process is not efficient or scalable, and it does not work well if more than one person (i.e., you) is contributing to your code. It is therefore more common to use a CI/CD service to implement CI/CD. These services essentially do what we described above but in an automated manner; we define a workflow which these services will automatically run at certain "trigger events" which we can also define (for example, making a pull request to the main branch of a GitHub repository might trigger the automatic deployment of a new version of the software).

There are many CI/CD services out there - the one we'll be using in this book is [GitHub Actions](https://docs.github.com/en/actions), which is easy to implement and set up directly in a GitHub repository. We won't focus too much on the specific syntax of GitHub Actions, but rather, we aim to show the general CI/CD workflow and illustrate the kind of things that are possible for you to set up.

````{note}
In this chapter we'll create the files needed to configure CI/CD workflows with GitHub Actions from scratch to clearly demonstrate the process step-by-step. However, the `cookiecutter` [template] we used to set up our package in **Chapter 3: {ref}`03:How-to-package-a-Python`** can make the files for you. Recall from that chapter that we chose to **not** include GitHub Actions in our package template:

```console
Select include_github_actions:
1 - no
2 - build
3 - build+deploy
Choose from 1, 2, 3 [1]: 1
```

In the future, feel free to include the workflow file(s) in your initial `cookiecutter` package template by choosing a different option (which option you should choose will become clear after reading this chapter).
````

## Continuous integration

Continuous integration (CI) refers to the process of continuously evaluating your code as it is updated, to try and catch any potential issues your updates have caused before they get to users. The CI process may include many of workflows we've seen throughout this book, such as code tests, calculation of code coverage, documentation building, among others. There are plenty of good resources available if you wish to learn more about CI, for example, the GitHub Actions [documentation](https://docs.github.com/en/actions/building-and-testing-code-with-continuous-integration/about-continuous-integration). In the remainder of this section, we'll implement CI on the `partypy` Python package we've been developing throughout this book.

### Set up

As discussed in **Chapter 4: {ref}`04:Package-structure-and-distribution`** and **Chapter 7: {ref}`07:Releasing-and-versioning`**, the first thing we should do when preparing to make changes to our packaging project is create a new version control branch to work on. Before we do that, we'll also make sure we're on the main branch and that we've pulled all recent changes to our local version of our project:

```{prompt} bash \$ auto
$ git checkout main
$ git pull
$ git checkout -b ci-cd
```

```console
Switched to a new branch 'ci-cd'
```

To setup CI with GitHub Actions we first need to add a "workflow" file to our package which will define the workflow we want GitHub Actions to run on our behalf when we make changes to our package. GitHub Actions uses *.yml* files to specify workflow files and they should be added to a sub-directory in your root package directory named *`.github/workflows`*. Let's create a new file in that location called *`ci-cd.yml`* with the following commands entered in the terminal from the root directory of `partypy`:

```{prompt} bash \$ auto
$ mkdir -p .github/workflows
$ touch .github/workflows/ci-cd.yml
```

Your package directory structure should now look something like this:

```
partypy
├── .github
│   └── workflows
│       └── ci-cd.yml
├── .gitignore
├── .readthedocs.yml
├── CHANGELOG.md
├── CONDUCT.md
├── CONTRIBUTING.md
├── docs
├── LICENSE
├── pyproject.toml
├── README.md
├── src
└── tests
```

Open the new *`ci-cd.yml`* file in an editor of your choice. We are going to set up CI that triggers every time anybody makes a push or a pull-request to the "main" branch of our repository. To set this up, copy and paste the following text into *`ci-cd.yml`*:

```yaml
name: ci-cd

on:
  push:
    branches:
      - main
  pull_request:
    branches:
      - main
```

```{note}
As mentioned earlier, we won't discuss the specific syntax of GitHub Actions *.yml* files in too much detail here, but instead, refer readers to the excellent GitHub Actions [documentation](https://docs.github.com/en/actions/reference/workflow-syntax-for-github-actions) for more details on workflow file syntax.
```

Now we need to set up the workflow that will trigger if one of the above events occurs. GitHub Actions essentially provides you with a blank operating system (of your choice) which we need to set up before we can do things like run our package's tests or check that we can build its documentation. Our set up will involve the following steps:

1. Set up operating system;
2. Set up Python;
3. "Check out" repository so we can access its contents;
4. Install `poetry`; and, 
5. Use `poetry` to install `partypy`.

One of the great things about GitHub Actions is that it supports a "Marketplace" where other developers create and share "actions" that we can leverage in our workflow, saving us time and effort. If you need to do something in your workflow, like install `poetry`, the chances are that somebody has already created an action for that which contains all the code needed to install `poetry` and which we we can use in our workflow without having to write the code ourselves. We'll be using several actions to help with our set up:

1. `actions/checkout@v2`: this action checks-out our repository so the GitHub Actions workflow can access the files in it.
2. `actions/setup-python@v2`: this action sets up a Python environment.
3. `snok/install-poetry@v1`: this action installs and sets up `poetry`.

For our example, we are going to set up a workflow for `partypy` that uses the `Ubuntu` operating system and Python version 3.9. A workflow run is made up of one or more "jobs" which contain a sequence of tasks to execute called "steps". We'll call our first job "ci" and will fill it with the five steps we outlined above to set up our workflow:

```yaml
name: ci-cd

on:
  push:
    branches:
      - main
  pull_request:
    branches:
      - main

jobs:
  ci:
    # Step 1. Set up operating system
    runs-on: ubuntu-latest
    steps:
    # Step 2. Set up Python environment
    - name: Set up Python 3.9
      uses: actions/setup-python@v2
      with:
        python-version: 3.9
    # Step 3. Check-out repository so we can access its contents
    - name: Check-out repository
      uses: actions/checkout@v2
    # Step 4. Install poetry
    - name: Install poetry
      uses: snok/install-poetry@v1
      with:
        version: 1.2.0a2
    # Step 5. Install our partypy package
    - name: Install package
      run: poetry install
```

```{note}
The meaning of the keywords in the workflow above are as follows:

- `name`: a name for the step to display on GitHub.
- `uses`: selects an action to run as part of a step.
- `with`: a map of the input parameters for the selected action.
- `run`: commands to run at the command-line.
```

Take a moment to read through the workflow above to get a high-level understanding of what we're doing here. Essentially, GitHub Actions is providing us with a blank operating system which we are setting up to be able to evaluate our package (we effectively did most of these steps on our local computer back in **Chapter 2: {ref}`02:System-setup`**).

With our system set up we can now populate our CI pipeline with the actions we want GitHub Actions to perform for us, in our case that will be:
1. Checking the style of our Python code;
2. Running `partypy`'s unit tests;
3. Recording the test coverage; and,
4. Checking that `partypy`'s documentation builds correctly.

Each of these steps is described in more detail below.

### Style checking

The first thing we want to check is that any new code adheres to a style guide. We haven't discussed code style in this book yet, but it's useful to know about. Code style is about making your code adhere to a set of guidelines in an effort to make it as readable and consistent as possible. Remember, "Code is read much more often than it is written".

The Python Style Guide is outlined in [PEP 8](https://www.python.org/dev/peps/pep-0008/). It is also important when sharing and collaborating on your code with other users (including your future self!), to ensure that code is written in a uniform way across the package. It is worth taking the time to read through the PEP 8 style guidelines, but here are a few highlights:

- Indent using 4 spaces;
- Have white space around operators, e.g. `x = 1` not `x=1`;
- But avoid extra white space, e.g. `f(1)` not `f (1)`;
- Variable and function names use `underscores_between_words`; and,
- Much more...

Luckily, you don't have to remember all these guidelines as there are many tools out there to help you! [flake8](https://flake8.pycqa.org/en/latest/#) is one of the most popular style guide checking tools and we'll use it to check style in our code here. First, we'll add `flake8` as a development dependency to our `partypy` package:

```{prompt} bash \$ auto
$ poetry add --dev flake8
```

We can now check that our code conforms to `flake8` by using the following command from our package's root directory:

```{prompt} bash \$ auto
$ flake8 .
```

```console
./tests/test_partypy.py:36:26: W292 no newline at end of file
./src/partypy/simulate.py:37:80: E501 line too long (83 > 79 characters)
./src/partypy/__init__.py:10:56: W292 no newline at end of file
```

```{note}
In the command above we are pointing `flake8` to our package's entire directory with the `.` syntax, within which it will search for and assess every *.py* file. You can also choose to point `flake8` only to a specific file, e.g., `flake8 ./src/partypy/simulate.py`.
```

In the output above we can see that `flake8` noticed several style violations (if you've been following along with the `partypy` example in this book, the violations you see might still be slightly different to those above depending on how you formatted your code). These violations can be fixed by opening the code files in an editor and addressing the issues, however we'll refrain from doing that for now, so we can demonstrate what a failed CI workflow looks like later on.

```{tip}
Note that `flake8` does not format your code for you, it only scans it for style and provides warnings. However, tools that actually format your code to adhere to a style guide do exist. One of the most commonly used formatters at the moment is [black](https://black.readthedocs.io/en/stable/).
```

We can include this `flake8` testing in our CI pipeline by adding the following code as a step in our *`ci.yml`* file:

```yaml
    # Step 6. Check Python code style
    - name: Check style
      run: poetry run flake8 .
```

Now, every time somebody pushes code updates or makes a pull request to the "main" branch of our repository, the code will be checked using `flake8`.

```{attention}
Once we've added a few more steps to our CI pipeline we'll go and see it all in action on GitHub!
```

### Running tests

Remember all the hard work we put into writing tests for our package back in the **Chapter 5: {ref}`05:Testing`**? Well, we likely want to make sure that these tests (and any others that we add) continue to pass for any new updates to our code. Just like we did with `flake8` we can automatically run our tests every time somebody pushes code updates or makes a pull request to our repository. The set up here is pretty easy! Recall that we used `pytest` as our testing framework, and this is listed as a development dependency for our package so will already be installed by our CI workflow in the "Install dependencies" step. Therefore, we just need to add the `pytest` command as a step in our *`ci.yml`* file:

```yaml
    # Step 7. Run unit tests for partypy
    - name: Test with pytest
      run: poetry run pytest --cov=./ --cov-report=xml
```

Note that in the command above we are also asking for our test coverage through the `--cov` argument and outputting a report to *.xml* format, which requires `pytest-cov` which we also used and added as a dependency of our package in **Chapter 5: {ref}`05:Testing`**. We will use the outputted report to automatically extract and record the test coverage for our package in the next section.

### Recording code coverage

In the previous step we ran the unit tests for our package. An important part of the testing workflow is evaluating and keeping a record of our test's code coverage. There are quite a few services out there for helping you do this, but we're going to use the free service of [Codecov](https://codecov.io/). We're also going to leverage a [pre-made GitHub Action workflow](https://github.com/marketplace/actions/codecov) provided by Codecov to help us record our test coverage. All that is required is to add the following step to our *`ci.yml`* file:

```yaml
    # Step 8. Record code coverage
    - name: Upload coverage to Codecov  
      uses: codecov/codecov-action@v2
      with:
        file: ./coverage.xml
        fail_ci_if_error: true
```

```{attention}
If your GitHub repository is public, then no further action is needed. However if you're repository is private, you'll need to provide an "upload token" in your repository settings as described in the [Codecov GitHub Action documentation](https://github.com/marketplace/actions/codecov). 
```

### Build documentation

The final step in our CI workflow will be to check that our documentation builds without issue. We'll simply use the same `sphinx` command we used back in **Chapter 4: {ref}`04:Package-structure-and-distribution`** to do this:

```yaml
    # Step 9. Build partypy documentation
    - name: Build documentation
      run: poetry run make html -C docs
```

### Testing the CI workflow

Nice work! We've set up our CI pipeline. Your final *`.github/workflows/build.yml`* file should look like this:

```yaml
name: ci-cd

on:
  push:
    branches:
      - main
  pull_request:
    branches:
      - main

jobs:
  ci:
    # Step 1. Set up operating system
    runs-on: ubuntu-latest
    steps:
    # Step 2. Set up Python environment
    - name: Set up Python 3.9
      uses: actions/setup-python@v2
      with:
        python-version: 3.9
    # Step 3. Check-out repository so we can access its contents
    - name: Check-out repository
      uses: actions/checkout@v2
    # Step 4. Install poetry
    - name: Install poetry
      uses: snok/install-poetry@v1
      with:
        version: 1.2.0a2
    # Step 5. Install our partypy package
    - name: Install package
      run: poetry install
    # Step 6. Check Python code style
    - name: Check style
      run: poetry run flake8 .
    # Step 7. Run unit tests for partypy
    - name: Test with pytest
      run: poetry run pytest --cov=./ --cov-report=xml
    # Step 8. Record code coverage
    - name: Upload coverage to Codecov  
      uses: codecov/codecov-action@v2
      with:
        file: ./coverage.xml
        fail_ci_if_error: true
    # Step 9. Build partypy documentation
    - name: Build documentation
      run: poetry run make html -C docs
```

We're now ready to test out our workflow! Let's go ahead and commit our changes to version control and push to GitHub:

```{prompt} bash \$ auto
$ git add pyproject.toml poetry.lock
$ git commit -m "test: add flake8 as dev dependency"
$ git add .github/workflows/ci-cd.yml
$ git commit -m "feat: add CI workflow"
$ git push -u origin ci-cd
```

We have configured our CI pipeline to trigger when someone makes a pull request with the "main" branch. So to trigger our CI, let's open a pull request between our "ci-cd" branch and the "main" branch:

```{figure} images/cicd-pull-request.png
---
width: 100%
name: 08-cicd-pull-request.png
alt: Making a pull request to merge changes from "ci" into "main".
---
Making a pull request to merge changes from "ci" into "main".
```

Making that pull request will automatically trigger our CI pipeline with GitHub Actions, as you can on the newly opened pull request page:

```{figure} images/.png
---
width: 100%
name: 08-
alt: Triggered CI workflow.
---
Triggered CI workflow.
```

We expect our workflow to fail because we didn't fix our `flake8` style issues from earlier. We can click on one the workflow to investigate further:

```{figure} images/.png
---
width: 100%
name: 08-
alt: Failing CI due to code not adhering to flake8 style.
---
Failing CI due to code not adhering to flake8 style.
```

We can go back and fix those style violations locally and then commit and push those changes to GitHub, to trigger the CI workflow again. This time, we'd hope to see a passing workflow:

```{figure} images/.png
---
width: 100%
name: 08-
alt: Successful CI.
---
Successful CI.
```

Don't merge that pull request just yet. In the next section we'll develop our CD pipeline and add it to the pull request.

## Continuous deployment

Whereas CI verifies that your updated code is working as expected, Continuous Deployment (CD) takes that updated code and deploys it into production. In the case of Python packaging, that typically means building and pushing an updated package version to PyPI. In the **Chapter 7: {ref}`07:Releasing-and-versioning`** we discussed how to version and release a Python package. Here, we are going to automate this process with a GitHub Actions workflow.

### Set up

We'll continue working on our "ci-cd" branch from earlier. Our aim here is to add a job to our existing workflow file that will trigger a deployment of our package each time updated code is pushed to the "main" branch of our repository. We only want this job to execute if:

1. the "ci" job before it passes - we don't want to deploy a new version of our package if it isn't passing its tests; and,
2. code is pushed to the "main" branch - we don't want to deploy a new version of our package when a pull request is opened, we only want to deploy a new version when a pull request is merged into the "main" branch or the "main" branch is modified directly.

We can set up a new job and add the two constraints above with the following syntax:

```yaml
name: ci-cd

on:
  push:
    branches:
      - main
  pull_request:
    branches:
      - main

jobs:
  ci:
    # ...
    # CI steps hidden
    # ...
  cd:
    # Only run this job if the "ci" job passes
    needs: ci
    # Only run this job if new work is pushed to "main"
    if: github.event_name == 'push' && github.ref == 'refs/heads/main'
```

Now we can setup our CD workflow. in GitHub Actions, each job runs in a fresh virtual environment so we need to set up our system for each new job. Following that, our CD workflow will effectively comprise all the steps we walked through manually in **Chapter 7: {ref}`07:Releasing-and-versioning`**:

1. Set up operating system;
2. Set up Python;
3. "Check out" repository so we can access its contents;
4. Install `poetry`;
5. Create a new version of `partypy`;
6. Use `poetry` to build source and wheel distributions for new version of `partypy`;
7. Upload new version to TestPyPI;
8. Test that new version installs successfully from TestPyPI; and,
10. Upload new version to PyPI.

Steps 1-4 are the same as we set up previously for our CI workflow:

```yaml
name: ci-cd

on:
  push:
    branches:
      - main
  pull_request:
    branches:
      - main

jobs:
  ci:
    # ...
    # CI steps hidden
    # ...
  cd:
    # Only run this job if the "ci" job passes
    needs: ci
    # Only run this job if the "main" branch changes
    if: github.event_name == 'push' && github.ref == 'refs/heads/main'
    # Step 1. Set up operating system
    runs-on: ubuntu-latest
    steps:
    # Step 2. Set up Python environment
    - name: Set up Python 3.9
      uses: actions/setup-python@v2
      with:
        python-version: 3.9
    # Step 3. Check-out repository so we can access its contents
    - name: Check-out repository
      uses: actions/checkout@v2
    # Step 4. Install poetry
    - name: Install poetry
      uses: snok/install-poetry@v1
      with:
        version: 1.2.0a2
```

We'll discuss steps 5-10 in more detail in the sections below.

### Creating a new package version

As we went through in **Chapter 7: {ref}`07:Releasing-and-versioning`**, to create a new release of our package we typically do a few key things:

1. Bump the package version depending on whether we have a patch, minor, or major release;
2. Document what's changed in the new release in the *`CHANGELOG.md`*; and,
3. Create a new release on GitHub (or other hosting service, if you're using one).

We are going to use the [Python Semantic Release](https://python-semantic-release.readthedocs.io/en/latest/) (PSR) tool to help us automate these steps. Put simply, PSR is able to parse commit messages to determine what the next version of the package should be. The idea is to used a standardized commit message format and syntax which PSR can parse to determine how to increment the version number. The parser is customizable but follows the [Angular commit style](https://github.com/angular/angular.js/blob/master/DEVELOPERS.md#commit-message-format) by default, which we've actually been using throughout this book. In this style if a commit message is prefixed with "fix" or "perf" (performance) a patch version bump will be made. If a commit message is prefixed with "feat" (feature) a minor bump will be made. If a message contains "BREAKING CHANGE:" a major bump will occur. There are other commit message types too, such as "doc" and "style" which don't be default result in a version bump (but can be configured to do so if desired).

The PSR tool is not only able to automatically bump our package version based on commit messages, but it can also automatically update our *`CHANGELOG.md`* using the commits since the last release and make a new release of our source on GitHub. We'll add PSR as a step in our CD workflow shortly but first, we'll configure it from our *`pyproject.toml`* file by adding the following section:

```toml
[tool.semantic_release]
version_variable = "pyproject.toml:version" # tells PSR where to find and update package version
branch = "main"                             # branch of repo to generate releases from
changelog_file = "CHANGELOG.md"             # the file we want PSR to update with changes
build_command = false                       # don't build package (we'll do this later)
upload_to_pypi = false                      # don't upload to PyPI (we'll do this later)
upload_to_release = false                   # don't attach built distributions to GitHub release
```

We've added comments above to provide clarity on what each line is doing and you can read more about these different configuration options in the [PSR documentation](https://python-semantic-release.readthedocs.io/en/latest/configuration.html). As you can see, PSR is able to build and upload your package to PyPI automatically as well, but we chose to disable this functionality because in our CD workflow we are going to build our package ourselves and first upload to TestPyPI, make sure we can install it correctly from there, and only then will we upload to PyPI.

With PSR configured, we can now add it as a step to our CD workflow:

```yaml
    # Step 5. Use PSR to increment version
    - name: Python Semantic Release
      run: |
          pip install python-semantic-release --quiet
          git config user.name github-actions
          git config user.email github-actions@github.com
          semantic-release publish -v DEBUG
```

In the code above we first install the PSR tool with `pip`, we then need to configure the `git` credentials on the machine because PSR will be making commits to our repository and credentials (it will be changing files likes *`pyproject.toml`* and *`CHANGELOG.md`*) and credentials (name and email) are required for this, and finally, we use PSR with `semantic-release publish` to execute our versioning workflow (the optional `-v DEBUG` argument tells PSR to print information to the screen as it's executing).

We'll see PSR in action shortly, but let's first configure the rest of our workflow file.

### Build source and wheel distributions

With a new version of our package created, we now need to build new source and wheel distributions using the same `poetry build` command we've used throughout this book:

```yaml
    # Step 6. Build package distributions
    - name: Build source and wheel distributions
      run: poetry build
```

### Uploading to TestPyPI and PyPI

With our new package distributions built, we can publish our package to TestPyPI and try to install it, to make sure everything is working as expected. This step is not strictly necessary but it's a good idea because it can help catch any unexpected errors before releasing your new package publicly on PyPI.

Rather than write the code needed to do all this from scratch, we'll leverage the [gh-action-pypi-publish](https://github.com/pypa/gh-action-pypi-publish) action from the GitHub Actions marketplace. To use this action, we can add the following step to our CD workflow:

```yaml
    # Step 7. Publish to TestPyPI
    - name: Publish to TestPyPI
      uses: pypa/gh-action-pypi-publish@release/v1
      with:
        user: __token__
        password: ${{ secrets.TEST_PYPI_API_TOKEN }}
        repository_url: https://test.pypi.org/legacy/
```

As you can see, this action relies on token authentication with TestPyPI (rather than the classic username and password authentication). To use the action, you'll need to log-in to [TestPyPI](https://test.pypi.org), [create an API token](https://pypi.org/help/#apitoken), and [add the token as a secret](https://docs.github.com/en/actions/reference/encrypted-secrets) called `TEST_PYPI_API_TOKEN` to your GitHub repository.

The above action will publish the new version of your package to TestPyPI. We now want to test that we can install the package correctly from TestPyPI using the same command we used back in **Chapter 3: {ref}`03:How-to-package-a-Python`**:

```yaml
    # Step 8. Test install from TestPyPI
    - name: Test install from TestPyPI
      run: |
          pip install \
          --index-url https://test.pypi.org/simple/ \
          --extra-index-url https://pypi.org/simple \
          partypy
```

We can now add the final step to our CD workflow which will be to publish our new package version to PyPI. This uses the same action as earlier, and will require you to obtain a token from PyPI and add the token as a called `PYPI_API_TOKEN` to your GitHub repository.

```yaml
    # Step 9. Publish to PyPI
    - name: Publish to PyPI
      uses: pypa/gh-action-pypi-publish@release/v1
      with:
        user: __token__
        password: ${{ secrets.PYPI_API_TOKEN }}
```

### Testing the CD workflow

We've now set up our CD pipeline. It took a bit of time and a few new tools, but what we have now is an automated way to version and deploy our package, which will save a lot of time and effort in the future. You final `cd` job in *`ci-cd.yml`* workflow file should look like this:

```yaml
name: ci-cd

on:
  push:
    branches:
      - main
  pull_request:
    branches:
      - main

jobs:
  ci:
    # ...
    # CI steps hidden
    # ...
  cd:
    # Only run this job if the "ci" job passes
    needs: ci
    # Only run this job if the "main" branch changes
    if: github.event_name == 'push' && github.ref == 'refs/heads/main'
    # Step 1. Set up operating system
    runs-on: ubuntu-latest
    steps:
    # Step 2. Set up Python environment
    - name: Set up Python 3.9
      uses: actions/setup-python@v2
      with:
        python-version: 3.9
    # Step 3. Check-out repository so we can access its contents
    - name: Check-out repository
      uses: actions/checkout@v2
      with:
        fetch-depth: 0
    # Step 4. Install poetry
    - name: Install poetry
      uses: snok/install-poetry@v1
      with:
        version: 1.2.0a2
    # Step 5. Use PSR to increment version
    - name: Use Python Semantic Release to bump version and tag release
      run: |
          pip install python-semantic-release --quiet
          git config user.name github-actions
          git config user.email github-actions@github.com
          semantic-release publish
    # Step 6. Build package distributions
    - name: Build source and wheel distributions
      run: poetry build
    # Step 7. Publish to TestPyPI
    - name: Publish to TestPyPI
      uses: pypa/gh-action-pypi-publish@release/v1
      with:
        user: __token__
        password: ${{ secrets.TEST_PYPI_API_TOKEN }}
        repository_url: https://test.pypi.org/legacy/
    # Step 8. Test install from TestPyPI
    - name: Test install from TestPyPI
      run: |
          pip install \
          --index-url https://test.pypi.org/simple/ \
          --extra-index-url https://pypi.org/simple \
          partypy
          pip show partypy | grep Version
    # Step 9. Publish to PyPI
    - name: Publish to PyPI
      uses: pypa/gh-action-pypi-publish@release/v1
      with:
        user: __token__
        password: ${{ secrets.PYPI_API_TOKEN }}
 
```

Let's go ahead and commit our changes to version control and push to GitHub:

```{prompt} bash \$ auto
$ git add .github/workflows/ci-cd.yml
$ git commit -m "feat: add CD workflow"
$ git push
```

Upon pushing this change, we have updated the pull request between the "ci-cd" branch and the "main" branch which triggers our CI/CD workflow. However, because this is a pull request, and we haven't made any changes directly to the "main" branch yet, the `cd` job of our workflow will not be executed:

```{figure} images/.png
---
width: 100%
name: 08-
alt: Triggered CI workflow.
---
Triggered CI workflow.
```

That part of the workflow will only be executed upon merging the pull request, which we'll do now. Once merged, the workflow file will be triggered (merging a pull request is akin to pushing new code to "main"), so we can navigate to the "Action" tab of our GitHub repository and view the workflow. This time the `cd` job will be executed:

```{figure} images/.png
---
width: 100%
name: 08-
alt: Triggered CI workflow.
---
Triggered CI workflow.
```

Investigating the workflow log, we can see that PSR parsed the commit messages since the previous release, and determined that our package should be bumped with a minor release:

```{figure} images/.png
---
width: 100%
name: 08-
alt: Triggered CI workflow.
---
Triggered CI workflow.
```

## An example of CI/CD in action

To finish off this chapter, we'll go through the process of updating our `partypy` package one more time. We are going to add a brand new feature to our package; the option to add confidence intervals to the histogram plots output by `plotting.plot_simulation()`:

```{figure} images/.png
---
width: 100%
name: 08-
alt: Triggered CI workflow.
---
Triggered CI workflow.
```

To do that, we'll need to do the following steps:

1. Create a new development branch
2. Update source code
3. Update docs
4. Update tests
5. Push changes to GitHub to trigger CI/CD

```{prompt} bash \$ auto
$ git checkout main
$ git pull
$ git checkout -b feat/add-conf-int
```

```console
Switched to a new branch 'feat/add-conf-int'
```

Change plotting.py to:

```python
import pandas as pd
import altair as alt


def _quantiles(results, C=0.95):
    """Calculate quantiles of simulation results.

    Parameters
    ----------
    results : pandas.DataFrame
        DataFrame of simulation results from `partpy.simulate_party()`
    C : float, optional
        Confidence level, between 0 and 1. By default, 0.95.
    """
    if not 0 < C < 1:
        raise ValueError("ci must be 0 < ci < 1.")
    lower_q = results.quantile(0.5 - C / 2).to_numpy()
    upper_q = results.quantile(0.5 + C / 2).to_numpy()
    return lower_q, upper_q


def plot_simulation(results, C=None):
    """Plot a histogram of simulation results.

    Parameters
    ----------
    results : pandas.DataFrame
        DataFrame of simulation results from `partpy.simulate_party()`
    C : float, optional
        Confidence level, between 0 and 1. If provided, confidence intervals
        will be displayed on chart. By default, 0.95.

    Returns
    -------
    altair.Chart
        Histogram of simulation results.

    Examples
    --------
    >>> from partypy.simulate import simulate_party
    >>> from partypy.plotting import plot_simulation
    >>> results = simulate([0.1, 0.5, 0.9])
    >>> plot_simulation(results)
    altair.Chart
    """

    histogram = (
        alt.Chart(results)
        .mark_bar()
        .encode(
            x=alt.X(
                "Total guests",
                bin=alt.Bin(maxbins=30),
                axis=alt.Axis(format=".0f"),
            ),
            y="count()",
            tooltip="count()",
        )
    )

    if C is not None:
        lower_q, upper_q = _quantiles(results, C)
        quantiles = (
            alt.Chart(
                pd.DataFrame({"quantiles": [lower_q, upper_q]})
            )
            .mark_rule(color="red", strokeWidth=3)
            .encode(x="quantiles")
        )
        return histogram + quantiles
    else:
        return histogram

```

Commit

```{prompt} bash \$ auto
$ git add src/partypy/plotting.py
$ git commit -m "feat: add confidence interval option to plot_simulation"
```

Add new test:

```python
def test_plot_simulation_ci(test_data):
    results = simulate_party(test_data["p_0"])
    plot = plot_simulation(results, C=0.95)
    assert isinstance(plot, alt.LayerChart)
    assert plot.layer[0].mark == "bar"
    assert plot.layer[0].data["Total guests"].sum() == 0
    with raises(ValueError):
        plot_simulation(results, C=0)
```

Commit

```{prompt} bash \$ auto
$ git add tests/test_partypy.py
$ git commit -m "test: add tests for confidence interval option"
```

Update usage docs:


Commit

```{prompt} bash \$ auto
$ git add docs/source/usage.ipynb
$ git commit -m "docs: add usage example of confidence interval option"
```

push to github, open PR and view CI pipelie:

>fig

merge pr and observe auto deployment:

>fig

Check out changelog and new version

Overall, CI/CD is a great way to streamline your package development and open-source collaboration. In this chapter we've walked through a simple CI/CD workflow for a Python package using tools like `poetry`, GitHub Actions, and Python Semantic Release. These, and other, tools can be configured in many different ways to achieve almost any workflow imaginable!